In [1]:
import tensorflow as tf 
import tensorflow_datasets as tfds
from tensorflow import keras

## Load the dataset


In [2]:
# Load the mnist dataset
train_ds, test_ds = tfds.load(
    "mnist",
    split = ["train", "test"],
    shuffle_files = True
)

In [4]:
def preprocess_fn(data):
    image = tf.cast(data['image'], tf.float32) / 255
    label = data['label']
    return (image, label)

train_ds = train_ds.map(preprocess_fn).batch(128).prefetch(tf.data.AUTOTUNE)
test_ds = test_ds.map(preprocess_fn).batch(128).prefetch(tf.data.AUTOTUNE)

## Define the model

In [7]:
input_shape = (28, 28, 1)
num_classes = 10

model = keras.Sequential(
    [
        keras.Input(shape = input_shape),
        keras.layers.Conv2D(32, kernel_size = (3, 3), activation = "relu"),
        keras.layers.MaxPooling2D(pool_size = (2, 2)),
        keras.layers.Conv2D(64, kernel_size = (3, 3), activation = "relu"),
        keras.layers.MaxPooling2D(pool_size = (2, 2)),
        keras.layers.Flatten(),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(num_classes, activation = "softmax"),
    ]
)

In [8]:
model.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(),
    optimizer = keras.optimizers.Adam(learning_rate = 0.001),
    metrics = [keras.metrics.SparseTopKCategoricalAccuracy()],
)

## Setup tracking/visualization tool

In [21]:
import mlflow
mlflow.set_tracking_uri(uri = "http://127.0.0.1:8080/")
mlflow.set_experiment("/mlflow-tf.keras-mnist_10")

2024/11/15 14:21:59 INFO mlflow.tracking.fluent: Experiment with name '/mlflow-tf.keras-mnist_10' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/403313648814821506', creation_time=1731660719319, experiment_id='403313648814821506', last_update_time=1731660719319, lifecycle_stage='active', name='/mlflow-tf.keras-mnist_10', tags={}>

## MLflow Auto Logging

In [22]:
mlflow.tensorflow.autolog()

In [23]:
model.fit(x=train_ds, epochs=3)

2024/11/15 14:22:23 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'bbae673c859c4068bc8674ca9f008b82', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current tensorflow workflow


Epoch 1/3
469/469 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.7130 - sparse_top_k_categorical_accuracy: 0.9513

2024/11/15 14:22:32 WARNING mlflow.utils.checkpoint_utils: Checkpoint logging is skipped, because checkpoint 'save_best_only' config is True, it requires to compare the monitored metric value, but the provided monitored metric value is not available.


469/469 ━━━━━━━━━━━━━━━━━━━━ 8s 14ms/step - loss: 0.7122 - sparse_top_k_categorical_accuracy: 0.9514
Epoch 2/3
468/469 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.1097 - sparse_top_k_categorical_accuracy: 0.9995

2024/11/15 14:22:38 WARNING mlflow.utils.checkpoint_utils: Checkpoint logging is skipped, because checkpoint 'save_best_only' config is True, it requires to compare the monitored metric value, but the provided monitored metric value is not available.


469/469 ━━━━━━━━━━━━━━━━━━━━ 7s 14ms/step - loss: 0.1097 - sparse_top_k_categorical_accuracy: 0.9995
Epoch 3/3
465/469 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0816 - sparse_top_k_categorical_accuracy: 0.9996

2024/11/15 14:22:45 WARNING mlflow.utils.checkpoint_utils: Checkpoint logging is skipped, because checkpoint 'save_best_only' config is True, it requires to compare the monitored metric value, but the provided monitored metric value is not available.


469/469 ━━━━━━━━━━━━━━━━━━━━ 7s 14ms/step - loss: 0.0816 - sparse_top_k_categorical_accuracy: 0.9996


2024/11/15 14:22:45 WARNING mlflow.tensorflow: Failed to infer model signature: could not sample data to infer model signature: tuple index out of range
2024/11/15 14:22:45 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.
2024/11/15 14:22:57 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/11/15 14:22:57 INFO mlflow.tracking._tracking_service.client: 🏃 View run unique-skink-644 at: http://127.0.0.1:8080/#/experiments/403313648814821506/runs/bbae673c859c4068bc8674ca9f008b82.
2024/11/15 14:22:57 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:8080/#/experiments/403313648814821506.


In [25]:
score = model.evaluate(test_ds)
print(f"Test loss: {score[0]:.4f}")
print(f"Test accuracy: {score[1]: .2f}")

79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0427 - sparse_top_k_categorical_accuracy: 1.0000
Test loss: 0.0438
Test accuracy:  1.00


## Log with MLflow callback

In [26]:
from mlflow.tensorflow import MLflowCallback

# Turn off autologging
mlflow.tensorflow.autolog(disable = True)

with mlflow.start_run() as run:
    model.fit(
        x = train_ds,
        epochs = 2,
        callbacks = [MLflowCallback(run)]
    )

Epoch 1/2
469/469 ━━━━━━━━━━━━━━━━━━━━ 7s 14ms/step - loss: 0.0678 - sparse_top_k_categorical_accuracy: 0.9998
Epoch 2/2
469/469 ━━━━━━━━━━━━━━━━━━━━ 8s 16ms/step - loss: 0.0590 - sparse_top_k_categorical_accuracy: 0.9998


2024/11/15 14:28:50 INFO mlflow.tracking._tracking_service.client: 🏃 View run agreeable-hound-193 at: http://127.0.0.1:8080/#/experiments/403313648814821506/runs/1bc5d0f6edee46458e31ab2071eaae91.
2024/11/15 14:28:50 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:8080/#/experiments/403313648814821506.


## Customize the MLflow callback

In [28]:
import math
from mlflow.tensorflow import MlflowCallback

# Create our own callback by subclassing `MlflowCallback`.
class MlflowCustomCallback(MlflowCallback):
    def on_epoch_end(self, epoch, logs=None):
        if not self.log_every_epoch:
            return
        loss = logs["loss"]
        logs["log_loss"] = math.log(loss)
        del logs["loss"]
        self.metrics_logger.record_metrics(logs, epoch)


In [29]:
with mlflow.start_run() as run:
    run_id = run.info.run_id
    model.fit(
        x = train_ds,
        epochs = 3,
        callbacks = [MlflowCustomCallback(run, log_every_epoch=True)]
    )

2024/11/15 14:34:38 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during autologging: MlflowCallback.__init__() got multiple values for argument 'log_every_epoch'


Epoch 1/3


2024/11/15 14:34:38 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during autologging: 'MlflowCustomCallback' object has no attribute 'log_every_n_steps'


  1/469 ━━━━━━━━━━━━━━━━━━━━ 14s 31ms/step - loss: 0.0191 - sparse_top_k_categorical_accuracy: 1.0000

2024/11/15 14:34:38 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during autologging: 'MlflowCustomCallback' object has no attribute 'log_every_n_steps'
2024/11/15 14:34:38 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during autologging: 'MlflowCustomCallback' object has no attribute 'log_every_n_steps'
2024/11/15 14:34:38 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during autologging: 'MlflowCustomCallback' object has no attribute 'log_every_n_steps'


...

2024/11/15 14:35:08 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during autologging: 'MlflowCustomCallback' object has no attribute 'log_every_n_steps'
2024/11/15 14:35:08 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during autologging: 'MlflowCustomCallback' object has no attribute 'log_every_n_steps'
2024/11/15 14:35:08 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during autologging: 'MlflowCustomCallback' object has no attribute 'log_every_n_steps'


467/469 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 0.0458 - sparse_top_k_categorical_accuracy: 0.9999

2024/11/15 14:35:08 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during autologging: 'MlflowCustomCallback' object has no attribute 'log_every_n_steps'
2024/11/15 14:35:08 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during autologging: 'MlflowCustomCallback' object has no attribute 'log_every_n_steps'
2024/11/15 14:35:08 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during autologging: 'MlflowCustomCallback' object has no attribute 'log_every_epoch'


469/469 ━━━━━━━━━━━━━━━━━━━━ 10s 21ms/step - loss: 0.0458 - sparse_top_k_categorical_accuracy: 0.9999


2024/11/15 14:35:08 INFO mlflow.tracking._tracking_service.client: 🏃 View run colorful-wasp-921 at: http://127.0.0.1:8080/#/experiments/403313648814821506/runs/365948bb0eb34ebd8785f0cf3174e317.
2024/11/15 14:35:08 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:8080/#/experiments/403313648814821506.
